In [1]:
import sys

sys.path.append('../../../')
import glob, os
import pandas as pd
from el_evaluation import *
import numpy as np


In [2]:
## data formatting

df_train = pd.read_csv("data/type_train.csv")
print(df_train.shape)
df_test = pd.read_csv("data/type_test.csv")
print(df_test.shape)
df_data = pd.concat([df_train, df_test])
df_data["Mention"] = df_data.Mention_label.str.split(';').str[0]
df_data['QuestionMention'] = df_data.Question + '--' + df_data.Mention
df_data.head()

EmptyDataError: No columns to parse from file

In [3]:
df_data = df_data.sort_values(by=['QuestionMention', 'Label'])
df_data.to_csv("data/type_train_test_sorted.csv")
df_data.shape

(49817, 7)

## ceated type_missing.csv

In [ ]:
# type_missing_questions   = """Which species does an elephant belong?
# In which ancient empire could you pay with cocoa beans?
# Which city has the most inhabitants?
# What is in a chocolate chip cookie?
# Which holidays are celebrated around the world? 
# Who is the tallest basketball player?
# Did Kaurism√§ki ever win the Grand Prix at Cannes?
# What does IYCM stand for?
# Which city has the least inhabitants?
# Which city has the oldest running metro?
# What is the bridge with the longest span?"""
# type_missing_questions = type_missing_questions.split('\n')
# pd.DataFrame([[a] for a in type_missing_questions]).to_csv("tmp.csv")
# print(len(type_missing_questions))

In [ ]:
from sklearn.model_selection import KFold

df_data = pd.read_csv("data/type_train_test_sorted.csv")
question_list = np.array(df_data.Question.unique())
questions_idxes = list(range(len(df_data.Question.unique())))

kf = KFold(n_splits=5)
import os
average_performance = []
fold_num = 1
for train_index, test_index in kf.split(questions_idxes):
#     args.output_file_name = "output/complex_kfolds_{}.csv".format(fold_num)
    train_ques_set = question_list[train_index]
    test_ques_test = question_list[test_index]

    df_train_split = df_data[df_data.Question.isin(train_ques_set)]
    df_test_split = df_data[df_data.Question.isin(test_ques_test)]
    
    # filter out the questions with single positive or many negatives in trianing set
    filtered_question_mentions = []
    for qm in df_train_split.QuestionMention.unique():
        df_ = df_train_split[df_train_split.QuestionMention == qm]
        if df_.Label.sum() == 0:
            filtered_question_mentions.append(qm)
        if df_.Label.sum() == 1 and df_.shape[0] == 1:
            filtered_question_mentions.append(qm)
#             print(df_.Label.values)
    df_train_split_filtered = df_train_split[~df_train_split.QuestionMention.isin(filtered_question_mentions)]
    df_train_split_filtered = df_train_split_filtered.sort_values(by=['QuestionMention', 'Label'])
#     print(list(df_train_split_filtered.Label.values))
    
    print("=========== fold num --", fold_num, "===========")
    print("Before filtering train set:")
    print('Train', 'QuestionMention pairs -', df_train_split.shape[0], 'Question -', df_train_split.Question.unique().shape[0])
    print("After filtering train set:")
    print('Train', 'QuestionMention pairs -', df_train_split_filtered.shape[0], 'Question -', df_train_split_filtered.Question.unique().shape[0])
    
    print( 'Test', 'QuestionMention pairs -', df_test_split.shape[0], 'Question -', df_test_split.Question.unique().shape[0])

# #     os.mkdir('data/qald-5-folds/split{}'.format(fold_num))
    
#     df_missing = pd.read_csv("data/missing.csv", header=None)
#     df_missing.columns = ['Unnamed:0', 'Question', 'Entities', 'Classes']
#     df_missing = df_missing[['Question', 'Entities', 'Classes']]
    df_test_split.to_csv('data/type-qald-5-folds/split{}/test_fold.csv'.format(fold_num))
    df_train_split.to_csv('data/type-qald-5-folds/split{}/train_fold.csv'.format(fold_num))
#     df_missing.to_csv('data/type-qald-5-folds/split{}/missing.csv'.format(fold_num))
#     print( 'Test + missing questions', 'question pairs -', df_test.shape[0]+df_missing.shape[0], 
#           'questions -', df_test.Question.unique().shape[0]+df_missing.Question.shape[0])
    fold_num += 1
